In [1]:
import os
import pandas as pd
import json
import time
from openai import OpenAI

# === Parametri principali ===
LABEL_COLUMN = "class"
model = ""
MODEL_NAME = model     # Cambia con il modello Ollama che preferisci
N_FEW_SHOT = 5
N_GEN_AT_TIME = 20

# === Inizializza client OpenRouter ===
client = OpenAI(
    base_url="https://openrouter.ai/api/v1",
    api_key="",
)

def call_openrouter(prompt: str, model: str = MODEL_NAME) -> str:
    """
    Interroga un modello tramite l'API OpenRouter e restituisce la risposta testuale.
    """
    try:
        start = time.time()
        completion = client.chat.completions.create(
            extra_headers={
                "HTTP-Referer": "https://tuo-sito.it",  # opzionale
                "X-Title": "Dataset Augmentation",      # opzionale
            },
            model=model,
            messages=[
                {"role": "user", "content": prompt}
            ],
        )
        elapsed = time.time() - start
        print(f"✅ Risposta ottenuta da {model} in {elapsed:.2f} secondi")
        return completion.choices[0].message.content
    except Exception as e:
        print("❌ Errore nella chiamata API OpenRouter:", e)
        return ""

def generate_with_llm(file, minority_examples: pd.DataFrame, n_generate: int) -> pd.DataFrame:
    """
    Chiede all'LLM di generare nuove tuple coerenti con la classe minoritaria.
    """
    examples_text = minority_examples.to_csv(index=False, sep=";")

    prompt = f"""
        Sei un assistente che genera nuovi esempi per bilanciare un dataset.
        Ti verranno fornite alcune tuple di riferimento (few-shot) appartenenti alla stessa classe.
        Genera {N_GEN_AT_TIME} nuove tuple realistiche e coerenti con la distribuzione dei dati.
        Restituisci SOLO un output CSV valido con lo stesso schema (stesse colonne, separatore ",").
        
        Esempi (classe minoritaria):
        {examples_text}
        
        Output CSV:
        """
    response = call_openrouter(prompt)

    # Salva la risposta in un file .txt
    os.makedirs("../../../../Downloads/SMOTE-CDNN/SMOTE-CDNN/llm", exist_ok=True)
    out_path = f"./LLM/{file.replace('.csv','')}_{MODEL_NAME.replace('/','-')}_{n_generate}_{n_generate+N_GEN_AT_TIME}.txt"
    with open(out_path, "w", encoding="utf-8") as text_file:
        text_file.write(response)

    '''
    # Se vuoi tentare di leggere il CSV direttamente:
    try:
        df_new = pd.read_csv(pd.compat.StringIO(response), sep=",")
        return df_new
    except Exception:
        print("⚠️ Non è stato possibile leggere la risposta come CSV, restituisco vuoto.")
        return pd.DataFrame()
    '''

def augment_dataset(file):
    print(f"\n📂 Analizzando dataset: {file}")
    df = pd.read_csv(file)

    if LABEL_COLUMN not in df.columns:
        print(f"  ❌ Colonna '{LABEL_COLUMN}' non trovata in {file}.")
        return

    # Conta le classi
    counts = df[LABEL_COLUMN].value_counts()
    majority_class = counts.idxmax()
    minority_class = counts.idxmin()

    print(f"  🔹 Classe maggioritaria: {majority_class} ({counts[majority_class]})")
    print(f"  🔹 Classe minoritaria: {minority_class} ({counts[minority_class]})")

    # Numero di nuove tuple da generare
    diff = counts[majority_class] - counts[minority_class]
    if diff <= 0:
        print("  ✅ Dataset già bilanciato. Nessuna generazione necessaria.")
        return

    print(f"  ➕ Genererò {diff} nuove tuple per bilanciare la classe minoritaria.")

    # Preleva 5 esempi della classe minoritaria per il few-shot
    few_shot = df[df[LABEL_COLUMN] == minority_class].sample(
        min(N_FEW_SHOT, counts[minority_class]), random_state=42
    )

    # Genera nuove tuple con OpenRouter
    for i in range(0, diff + 10, N_GEN_AT_TIME):
        print(f"\t Genero da {i} a {i + N_GEN_AT_TIME}")
        generate_with_llm(file, few_shot, i)

In [3]:
from tqdm import tqdm
csv_files = [f for f in os.listdir("../../../../Downloads/SMOTE-CDNN/SMOTE-CDNN/llm") if f.endswith(".csv")]
if not csv_files:
    print("⚠️ Nessun file CSV trovato nella directory corrente.")

csv_files_already_processed = [f.split("_deepseek")[0] for f in os.listdir(
    "../../../../Downloads/SMOTE-CDNN/SMOTE-CDNN/llm") if f.endswith(".txt")]
files_already_processed = set(csv_files_already_processed)

for file in tqdm(csv_files,total=len(csv_files)):
    print(file)
    if file.replace(".csv","") not in files_already_processed:
        augment_dataset(file)

  0%|                                                                                           | 0/28 [00:00<?, ?it/s]

abalone9-18.csv

📂 Analizzando dataset: abalone9-18.csv
  🔹 Classe maggioritaria: 0 (688)
  🔹 Classe minoritaria: 1 (42)
  ➕ Genererò 646 nuove tuple per bilanciare la classe minoritaria.
	 Genero da 0 a 20
✅ Risposta ottenuta da alibaba/tongyi-deepresearch-30b-a3b:free	 in 100.67 secondi
	 Genero da 20 a 40
✅ Risposta ottenuta da alibaba/tongyi-deepresearch-30b-a3b:free	 in 92.50 secondi
	 Genero da 40 a 60
✅ Risposta ottenuta da alibaba/tongyi-deepresearch-30b-a3b:free	 in 48.14 secondi
	 Genero da 60 a 80


  0%|                                                                                           | 0/28 [04:03<?, ?it/s]


KeyboardInterrupt: 